# Commands

In [ ]:
# oomstopper --no-tail tfidf ; killbill tfidf ; cd ~/twinews-logs ; jupython -o nohup-tfidf-$HOSTNAME.out --venv st-venv ~/Workspace/Python/Datasets/Twinews/twinews/models/tfidf.ipynb

# Init

In [ ]:
import os ; os.environ["CUDA_VISIBLE_DEVICES"] = ""

In [ ]:
isNotebook = '__file__' not in locals()

In [ ]:
TEST = isNotebook # isNotebook, True, False

In [ ]:
from systemtools.hayj import *
from systemtools.location import *
from systemtools.basics import *
from systemtools.file import *
from systemtools.printer import *
from nlptools.preprocessing import *
from nlptools.basics import *
from twinews.utils import *
from twinews.models.ranking import *

In [ ]:
from nltk.stem import WordNetLemmatizer 
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import TruncatedSVD

In [ ]:
logger = Logger(tmpDir('logs') + "/tfidf.log") if isNotebook else Logger("tfidf-" + getHostname() + ".log")
tt = TicToc(logger=logger)
tt.tic()

# Config

In [ ]:
config = \
{
    'splitVersion': 2,
    
    'maxUsers': 2 if TEST else None, # Sub-sampling
    'minDF': None if TEST else None, # 1 / 2000 or None
    'maxDF': None if TEST else None, # 300 or None
    
    'lowercase': False if TEST else False,
    'doLemmatization': False if TEST else False,
    'sublinearTF': True,
    
    'dimensions': None if TEST else 3000, # 20, 50, 100, 500, 3000, None (means no dimensionality reduction)
    'maxIter': 10, # 3, 10, 30, 100 (Iterations for SVD)
    
    'distance': 'cosine', # 'cosine', 'euclidean', 'kl', 'js'
    'historyRef': 0.3, # 1, 1.0, 0.6, 0.3, 3, 10, 30
}

In [ ]:
modelName = 'tfidf'

In [ ]:
if config['dimensions'] is None:
    del config['maxIter']

In [ ]:
# Here we check if we already generated ranking for this model with this specific config:
if not isNotebook:
    if rankingExists(modelName, config, logger=logger):
        raise Exception(modelName + " with this config already exist:\n" + b(config, 5))

# Getting data

In [ ]:
# Getting users and news
evalData = getEvalData(config['splitVersion'], maxExtraNews=0, maxUsers=config['maxUsers'], logger=logger)
(trainUsers, testUsers, trainNews, testNews, candidates, extraNews) = \
(evalData['trainUsers'], evalData['testUsers'], evalData['trainNews'],
 evalData['testNews'], evalData['candidates'], evalData['extraNews'])
bp(evalData.keys(), 5, logger)
log(b(evalData['meta'], 5), logger)

In [ ]:
# Here it is important to convert urls to lists because we want the same order to retrieve vectors by index...
# And we shuffle it so we do not stick urls a a user at the begin...
# But we seed the random to always have same order...
trainNewsList = shuffle(list(trainNews), seed=0)
testNewsList = shuffle(list(testNews), seed=0)
newsList = trainNewsList + testNewsList

In [ ]:
# Print all:
log(str(len(trainNewsList)) + " urls for trainNewsList", logger=logger)
log(str(len(testNewsList)) + " urls for testNewsList", logger=logger)
log(str(len(newsList)) + " urls for newsList", logger=logger)

In [ ]:
# We get sentences:
sentences = getNewsSentences(newsList, logger=logger)
bp(sentences, logger)

In [ ]:
# We flatten sentences:
for i in range(len(sentences)):
    sentences[i] = flattenLists(sentences[i])
docs = sentences
bp(docs, logger)

In [ ]:
# Lower case:
if config['lowercase']:
    for i in pb(list(range(len(docs))), logger=logger, message="Lower casing"):
        for u in range(len(docs[i])):
            docs[i][u] = docs[i][u].lower()
bp(docs, logger)

In [ ]:
# Lemmatization:
if config['doLemmatization']:
    lemmatizer = WordNetLemmatizer()
    pbar = ProgressBar(len(docs), logger=logger, message="Lemmatization")
    for i in range(len(docs)):
        for u in range(len(docs[i])):
            docs[i][u] = lemmatizer.lemmatize(docs[i][u])
        pbar.tic()
bp(docs, logger)

In [ ]:
# Filtering the corpus:
if config['minDF'] is not None or config['maxDF'] is not None:
    docs = filterCorpus(docs, minDF=config['minDF'], maxDF=config['maxDF'],
                        removeEmptyDocs=False, allowEmptyDocs=False, logger=logger)
    for doc in docs: assert len(doc) > 0
    bp(docs, logger)

In [ ]:
tt.tic("Data preprocessed")

# Infering vectors

In [ ]:
vectorizer = TfidfVectorizer\
(
    sublinear_tf=config['sublinearTF'],
    tokenizer=lambda x: x,
    preprocessor=lambda x: x,
    # lowercase=True, # Doesn't work because we erased preprocessor
)
vectors = vectorizer.fit_transform(docs)
assert vectors.shape[0] == len(newsList)

In [ ]:
tt.tic("TFIDF vectors generated")

# Dimensionality reduction

In [ ]:
if config['dimensions'] is not None:
    svd = TruncatedSVD(n_components=config['dimensions'], n_iter=config['maxIter'], random_state=42)
    vectors = svd.fit_transform(vectors)

# Making a dict url --> topic vector

In [ ]:
assert vectors.shape[0] == len(newsList)
urlsVectors = dict()
for i in range(len(newsList)):
    if isinstance(vectors, scipy.sparse.csr.csr_matrix):
        row = vectors.getrow(i)
    else:
        row = vectors[i]
    urlsVectors[newsList[i]] = row
bp(urlsVectors, logger)

# Ranking

In [ ]:
# Read the doc!
rankings = usersRankingsByHistoryDistance\
(
    trainUsers,
    candidates,
    config['historyRef'],
    urlsVectors,
    distanceMetric=config['distance'],
    logger=logger,
)

In [ ]:
bp(rankings, logger, 4)

In [ ]:
tt.tic("Rankings done")

# Adding rankings to the db

In [ ]:
# Read the doc!
addRanking(modelName, rankings, config, logger=logger)

In [ ]:
tt.tic("Rankings stored")

In [ ]:
totalDuration = tt.toc()

In [ ]:
notif(modelName + '-' + objectToHash(config)[:5] + " done in " + secondsToHumanReadableDuration(totalDuration) + " on " + getHostname())